In [1]:
"""
Configs:
    # Env
    env_id="Gridworld03-v01", size=3x3, action_reward=[-10, -12], terminal_reward=4
        - experiment on smaller q_vals between actions, by setting smaller terminal rewards 
        - once in the goal state, each action ends the episode and returns a reward of 5
        - no clear episodes, continuously train the agent with num_train_steps, 
            if the agent reaches terminal, just reset environment and keep training 
    # Params
    eps_sched_fn="poly(0.5)", lr_sched_fn="poly(0.8)"
    # Algos
    action_sigma=1, haver_delta=0.01, haver_const=varied
Status:
"""

from collections import defaultdict
import random
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")
sns.set_palette("tab20")
colors = sns.color_palette("bright")

import time
from tqdm import tqdm 
import multiprocessing

# import gymnasium as gym
import gym
import gym_examples
from gym.wrappers import FlattenObservation

from algos import *
from utils import *

In [ ]:

random.seed(123)
np.random.seed(123)
tdqm_disable = True

# params
env_id = "gym_examples/GridWorld03-v1"
gridworld_size = 3
max_steps = 20
gamma = 0.95

num_trials = 1000
num_steps_train = 10000
num_episodes_eval = 100

lr_sched_type = "poly"
lr_sched_fn = create_lr_sched_fn(lr_sched_type)

max_eps = 1.0
min_eps = 0.05
decay_rate = 0.0005
eps_sched_type = "poly"
eps_sched_fn = create_eps_sched_fn(eps_sched_type, min_eps, max_eps, decay_rate)

haver_const = 20.0
est_name = "haver" 
q_algo_name = "haver_q_learning"
q_algo = create_q_algo(q_algo_name)

# create gym env
env = gym.make(env_id, size=gridworld_size)
env_wrapped = FlattenObservation(env)
num_actions = env_wrapped.action_space.n
# print(f"num_actions = {num_actions}")
# print(env_wrapped.reset())
# stop

manager = multiprocessing.Manager()
# episode_lengths_list = manager.list()
episode_rewards_list = manager.list()
episode_start_vals_list = manager.list()

def run_trial(i_trial, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    # env = gym.make(env_id, size=gridworld_size)
    # env_wrapped = FlattenObservation(env)
    # env_wrapped.reset(seed=10000+i_trial)

    lr_sched_fn = create_lr_sched_fn(lr_sched_type)
    eps_sched_fn = create_eps_sched_fn(eps_sched_type, min_eps, max_eps, decay_rate)
    q_algo = create_q_algo(args["est_name"])

    Q_table, stats = q_algo(
        env_wrapped, num_actions, num_steps_train,
        gamma, lr_sched_fn, eps_sched_fn, tdqm_disable, args)

    episode_lengths, episode_rewards, episode_start_vals= zip(*stats)
    # episode_lengths_list.append(episode_lengths)
    episode_rewards_list.append(episode_rewards)
    episode_start_vals_list.append(episode_start_vals)



args = dict()
args["action_sigma"] = 1
args["haver_delta"] = 0.01
args["haver_const"] = 10.0
args["weightedms_num_data"] = 1000

pool = multiprocessing.Pool()

haver_const_ary = [1.0, 2.5, 5.0, 7.5, 10.0]
haver_name_ary = [f"haver_{x}" for x in haver_const_ary]

est_name_ary = ["weightedms", "double", "max"]
est_name_ary += haver_name_ary

# episode_lengths_mean_dict = defaultdict()
episode_rewards_mean_dict = defaultdict()
episode_start_vals_mean_dict = defaultdict()

for est_name in est_name_ary:
    start_time = time.time()
    print(f"\n-> est_name = {est_name}")
    if "haver" in est_name:
        args["est_name"] = "haver"
        args["haver_const"] = float(est_name.split("_")[-1])
    else:
        args["est_name"] = est_name
    
         
    pool.starmap(run_trial, [(i, args) for i in range(num_trials)])

    # episode_lengths_ary = np.hstack([episode_lengths_list])
    episode_rewards_ary = np.hstack([episode_rewards_list])
    episode_start_vals_ary = np.hstack([episode_start_vals_list])

    # episode_lengths_mean_dict[est_name] = np.mean(episode_lengths_ary, 0)
    episode_rewards_mean_dict[est_name] = np.mean(episode_rewards_ary, 0)
    episode_start_vals_mean_dict[est_name] = np.mean(episode_start_vals_ary, 0)
    # print(f"last_episode_length = {episode_lengths_mean_dict[est_name][-1]}")
    print(f"last_episode_reward_per_step = {episode_rewards_mean_dict[est_name][-1]:.4f}")
    print(f"last_episode_estim_start_val = {episode_start_vals_mean_dict[est_name][-1]:.4f}")
    
    # episode_lengths_list[:] = []
    episode_rewards_list[:] = []
    episode_start_vals_list[:] = []
    
    end_time = time.time()
    print(f"it takes {end_time-start_time:0.4f}")


-> est_name = weightedms
last_episode_reward_per_step = 0.4500
last_episode_estim_start_val = -0.9608
it takes 300.4695

-> est_name = double
last_episode_reward_per_step = 0.2230
last_episode_estim_start_val = -3.5480
it takes 186.2257

-> est_name = max
last_episode_reward_per_step = -1.6370
last_episode_estim_start_val = 6.5230
it takes 153.4133

-> est_name = haver_1.0


In [ ]:

fig, axes = fig, axes = plt.subplots(
        nrows=2, ncols=1, sharex=True, sharey=False, figsize=(8,8))
# axes = [axes]

x_ary = np.linspace(0, num_steps_train-1, num=10000, dtype=np.int32)
for est_name in est_name_ary:
    # axes[0].plot(x_ary, episode_rewards_mean_dict[est_name][x_ary], label=est_name)
    y_ary = running_avg(episode_rewards_mean_dict[est_name][x_ary], 200)
    axes[0].plot(
        x_ary, y_ary, label=est_name)
    axes[1].plot(x_ary, episode_start_vals_mean_dict[est_name][x_ary], label=est_name)
    # axes[2].plot(x_ary, episode_lengths_mean_dict[est_name][x_ary], label=est_name)

axes[0].set_title("reward_per_step")
axes[1].set_title("estim_qval_start_state")
# axes[2].set_title("episode_length")
axes[0].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[1].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# axes[2].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()